In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
# подсказка https://nagornyy.me/it/regressionnye-modeli-v-python/
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(X_train, y_train)
y_test_pred = slr.predict(X_test)
from sklearn.metrics import r2_score
print('R^2 test: {:.3f}'.format(r2_score(y_test, y_test_pred)))

R^2 test: 0.669


In [48]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
y_test_pred = ridge.predict(X_test)
print('R^2 test: {:.3f}'.format(r2_score(y_test, y_test_pred)))

R^2 test: 0.669


In [7]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_test_pred = lasso.predict(X_test)
print('Lasso R^2 test: {:.3f}'.format(r2_score(y_test, y_test_pred)))

R^2 test: 0.657


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [33]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html
from sklearn.linear_model import RidgeCV
clf = RidgeCV(alphas=[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4,1e+5]).fit(X_test, y_test)
print('RidgeCV R^2 test: {:.3f}'.format(clf.score(X_test, y_test)))

from sklearn.linear_model import LassoCV
clf = LassoCV(alphas=[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4,1e+5]).fit(X_test, y_test)
print('LassoCV R^2 test: {:.3f}'.format(clf.score(X_test, y_test)))

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1e-5,1e-4,1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4,1e+5]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)
print('GridSearchCV R^2 test: {:.3f}'.format(clf.score(iris.data, iris.target)))
# пока не разобрался почему не работает
# clf.fit(X_test, y_test)
# print('GridSearchCV R^2 test: {:.3f}'.format(clf.score(X_test, y_test)))

RidgeCV R^2 test: 0.766
LassoCV R^2 test: 0.720
GridSearchCV R^2 test: 0.993


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [56]:
from sklearn import preprocessing
X_scaled = preprocessing.scale(X_test)
X_scaler = preprocessing.StandardScaler().fit(X_test)
# y_scaler = preprocessing.StandardScaler().fit(y_test)
# print('StandardScaler R^2 test: {:.3f}'.format(scaler.score(X_test)))
print('scaler.mean_',X_scaler.mean_)
print('scaler.transform',X_scaler.transform(X_test))

X_minmaxscaler = preprocessing.MinMaxScaler().fit(X_test)
# print('minmaxscaler.mean_',minmaxscaler.mean_)
print('minmaxscaler.transform',minmaxscaler.transform(X_test))

scaler.mean_ [3.63094676e+00 1.05490196e+01 1.17377451e+01 5.88235294e-02
 5.47608824e-01 6.16083333e+00 6.86480392e+01 3.74294902e+00
 1.03137255e+01 4.24892157e+02 1.89990196e+01 3.58241275e+02
 1.34282353e+01]
scaler.transform [[-0.47678706 -0.44018634 -1.14921855 ... -1.33894103  0.41308415
  -0.60871618]
 [-0.48154797  1.2289217  -0.79642905 ... -0.78082095  0.42860582
  -1.3730385 ]
 [-0.47490641 -0.44018634  2.39213032 ...  0.61447927  0.3533257
   0.64388464]
 ...
 [ 1.40372123 -0.44018634  0.95107489 ...  0.67029128 -3.58319267
   1.08083841]
 [-0.48216228 -0.44018634 -1.08344424 ... -0.27851287  0.41851673
  -0.07882303]
 [-0.47633711 -0.44018634  0.16327799 ... -0.16688885  0.27704776
  -0.62258773]]
minmaxscaler.transform [[1.86839552e-03 0.00000000e+00 1.31598240e-01 ... 4.00000000e-01
  9.96412280e-01 1.80645161e-01]
 [1.09576391e-03 4.21052632e-01 2.18108504e-01 ... 5.11111111e-01
  1.00000000e+00 1.90615836e-02]
 [2.17360031e-03 0.00000000e+00 1.00000000e+00 ... 7.88888

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
y_test_pred = ridge.predict(X_test)
print('R^2 test: {:.3f}'.format(r2_score(y_test, y_test_pred)))

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [ ]:
data.head()

8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.